# 2 - How the backpropagation algorithm works 

In the last chapter we saw how neural networks can learn their weights and biases using the gradient descent algorithm. There was, however, a gap in our explanation: we didn't discuss how to compute the gradient of the cost function. In this chapter, we will learn a fast algorithm for computing such gradients, which known as $backpropagation$.

Backpropagation isn't just a fast alogrithm for learning. **It actually gives us detailed insights into how changing the weights and biases changes the overall behaviour of the network.** That's well worth study in detail. Let's enjoy it.

## 2.1 - Warm up: a fast matrix-based approach to compute the out from a neural network

Before discussing backpropagation, let's warm up with a fast matrix-based algorithm to compute the output from a neural network. In particular, this is a good way of getting comfortable with the notation used in backpropagation, in a familiar context. We acutally already saw this algorithm near the end of the last chapter (section 1.6), let's review it now.

We'll use $w^{l}_{jk}$ to denote the weight for the connection from the $k$-th neuron in the ($l$ - 1)-th layer to the $j$-th neuron in the $l$-th layer. So, for example, the diagram below shows the weight on a connection from the fourth neuron in the second layer to the second neuron in the third layer of a network:

![2.1](../picture/chapter2/p1.png)

We use a similar notation for the network's biases and activations. Explicitly, we use $b^{l}_{j}$ for the bias of the $j$-th neuron in the $l$-th layer. And we use $a^{l}_{j}$ for the activation of the $j$-th neuron in the $l$-th layer. The following diagram shows examples of these notations in use:

![2.1](../picture/chapter2/p2.png)

With these notations, **the activation $a^{l}_{j}$ of the $j$-th neuron in the $l$-th layer is related to the activations in the ($l$ - 1)-th layer by the equation**

$$a^{l}_{j} = \sigma(\sum_{k} w^{l}_{jk}a^{l-1}_{k} + b^{l}_{j}) , \tag{2.1}$$

where the sum is over all neurons $k$ in the ($l$ - 1)-th layer.

To rewrite this expression in a matrix form, we define a weight maxtrix $w^l$ for each layer, $l$. Similarly, for each layer $l$ we define a bias vector, $b^l$. And finally, we define an activation vector $a^l$ whose components are the activations $a^{l}_{j}$. The last ingredient we need to rewrite $2.1$ in a matrix form is the idea of vectorizing a function such as $\sigma$. The idea is that we want to apply a function such as $\sigma$ to every element in a vector $v$.

With these notations in mind, Equation 2.1 can be rewritten in the beautiful and compact vectorized form

$$a^{l} = \sigma(w^{l}a^{l-1} + b^{l}). \tag{2.2}$$

**This expression gives us a much more global way of thinking about how the activations in one layer relate to activations in the previous layer: we just apply the weight matrix to the activations, then add the bias vector, and finally apply the $\sigma$ function.**

When using Equation 2.2 to compute $a^l$, we compute the intermediate quantity $z^{l} \equiv w^la^{l-1} + b^l$ along the way. **This quantity turns out to be useful enough to be worth naming: we call $z^l$ the weighted input to the neurons in layer $l$. We'll make considerable use of the weighted input $z^l$ later in the chapter.**